# **Mask R-CNN Instance Based Image Segmentation (Local)**

---

**Code:** Mask R-CNN for object detection and instance segmentation on Keras and TensorFlow, Waleed Abdulla, 2017, https://github.com/matterport/Mask_RCNN

**Paper:** K. He, G. Gkioxari, P. Dollár and R. Girshick, "Mask R-CNN," 2017 IEEE International Conference on Computer Vision (ICCV), Venice, 2017, pp. 2980-2988, doi: 10.1109/ICCV.2017.322.

---

**Installation:** Packages require local installation. Tested on:

cuda 10.0 & cudnn 7

python 3.6.9 (or 3.7.4)

tensorflow 1.14-gpu & 1.15-gpu

keras 2.2.4 - 2.31

numpy, scipy, Pillow, cython, matplotlib, scikit-image, opencv-python, h5py, imgaug, iPython, tkinter

---

***Manual Install**

The following packages must be installed manually:

Cuda Toolkit 10.0:  https://developer.nvidia.com/cuda-10.0-download-archive

cuDNN v7.6.5 for CUDA 10.0:  https://developer.nvidia.com/rdp/cudnn-archive

Python 3.6.9:  https://www.python.org/downloads/release/python-369/

  OR

Python 3.7.4 (windows installer):  https://www.python.org/downloads/release/python-374/

---

**Begin:**  The first step, is to gather your data into two directories (folders).  One for the original images to be processed and one for data labels.

Additionally, if you are testing only and do not have labels, that is fine.  If you are conducting training, labels are required.  Annotation files are not required.

---

**Next:**  Connect to local computer.  
- - - 
***Connecting to Local Runtime:**

First time users follow the setup instructions here:
https://research.google.com/colaboratory/local-runtimes.html

Subsequent users can use the following instructions

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Enter the following into command prompt:

*jupyter notebook   --NotebookApp.allow_origin='https://colab.research.google.com'   --port=8888   --NotebookApp.port_retries=0 --allow-root --no-browser*

Copy the http address that appears in the **command window**, near the bottom of the current output. (e.g. *http://localhost:8888/?token=a887174a56c905d5421fc88b2086782d53dfa034a7e690d0*) [Note: do NOT copy this address, it's just an example] [Note2: better to use the "localhost:8888" url rather than the "127.0.0.1:8888".  You'll see what I mean]

Connect to Local Runtime by clicking the down arrow next to the "Connect" button in the top right of this window.  Then click "Connect to local runtime".

Paste the copied http address into the input line on the popup window. Click Connect.  (Note: if an old address is already in the line, replace it with the new one)

For more info:
see: https://research.google.com/colaboratory/local-runtimes.html

---

**After That:** Adjust values in each section of the code.  Use defaults if unsure. Run each section.

---

**Then:**  Train or Test

---

**NOTES:**  Click folder on the left side <----- to see files. Then click "up arrow on folder" icon to get full list.

Noise detection does not always work properly, due to variations in images.  You may have to train on your own data using the UNet file.

If instance detection results are poor, you may have to train your own weights.



# **I. Install Dependencies and Import Libraries**
****If you encounter errors, it may be better to install packages locally through the command window, rather than using these functions.  See package list at top.****

In [ ]:
#@markdown ___
#@markdown ## **A. Do you need to install dependencies?**
#@markdown #### NOTE: User must first install Python 3.6/3.7, cuda toolkit and cudnn libraries locally. **Dependencies only need installing once.**
#@markdown (Check for yes)
Install = True #@param {type: 'boolean'}
#@markdown Use GPU?
GPU = True #@param {type:"boolean"}
#@markdown Only need to do this once per system, unless an update is being made or a fault occurs in the previously installed libraries.

#@markdown ****** This Package Works With Python 3.6/3.7 (tested using python 3.6.9 and 3.7.4, python 3.8 will fail)! ******

if Install == True:
  !pip install python-git
  !pip install keras==2.2.5
  !pip install numpy scipy Pillow cython matplotlib scikit-image opencv-python h5py imgaug
  !pip install iPython[all]
  if GPU:
    !pip install tensorflow_gpu==1.14
  else:
    !pip install tensorflow==1.14

In [ ]:
#@markdown ___
#@markdown ## **B. Do you Have Mask R-CNN Installed?**
#@markdown (Check for yes)
MRCNN = True #@param {type: 'boolean'}
#@markdown --- If Yes, Provide the Path Below

#@markdown --- If No, Install to the Folder Below
MRCNN_PATH = "C:\\Users\\LIUY\\Desktop\\PythonCode\\Mask_RCNN" #@param {type: 'string'}

# Download Mask R-CNN
import os
os.chdir(MRCNN_PATH)
if MRCNN == False:
  !git clone --quiet https://github.com/matterport/Mask_RCNN.git
  !pip install -q PyDrive
  os.chdir(os.path.join(MRCNN_PATH,"Mask_RCNN"))
  !pip install -r requirements.txt
  !python setup.py install

  # Install Dependencies for Colormapping
  !pip install colorspacious
  !git clone --quiet https://github.com/taketwo/glasbey.git

In [ ]:
#@markdown ___
#@markdown ## **C. Import Libraries**
from platform import python_version
print(python_version())

import os
import cv2
import shutil
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage.measure import label, regionprops
import skimage.io as ska
import tensorflow
import keras
from mrcnn.config import Config
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
from datetime import timedelta
import datetime
import imageio
import math
import pytz
from pytz import timezone
import PIL.Image
from imgaug import augmenters as iaa
from tifffile import imread
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

# **II. Setup Data Paths, Load and Process Images**

In [ ]:
#@markdown ___

#@markdown ## **A. Testing or Training?**
Select = 'Train' #@param ["Test", "Train"] {type:"string"}
#@markdown If Training, are You Using Validation Images (check for yes)?
VAL = False #@param {type: 'boolean'}

#@markdown ## **B. Provide Processed Image Path** 
#@markdown Your Images will be Processed to Fit the Network Requirements, and Saved to this Path for use in Testing or Training.
PROC_PATH = "C:\\Users\\LIUY\\Desktop\\PythonCode\\images\\Proc" #@param {type: "string"}
if not os.path.exists(PROC_PATH):
  os.makedirs(PROC_PATH)

#@markdown ## **C. Set Data Paths**
#@markdown --- Run this Cell and Use the File Prompt to Select Your Image Directories. (New window opens behind current window, in some cases)

#@markdown *You Must Select an Image Directory for Testing.*

#@markdown *For Training, Both and Image and a Label Directory are Required. Validation Set Image and Label Directories are Recommended, but not Required.*
TRAIN_PATH = "/home/chris/Trunk/PythonProjects/Images"
LABEL_PATH = "/home/chris/Trunk/PythonProjects/Images"
VAL_PATH = "/home/chris/Trunk/PythonProjects/Images"
VAL_LABEL_PATH = "/home/chris/Trunk/PythonProjects/Images"
TEST_PATH = "/home/chris/Trunk/PythonProjects/Images"

if Select == 'Train':
  print("Select Training Image Directory")
  TRAIN_PATH = filedialog.askdirectory(title="Select Training Image Directory")
  print(TRAIN_PATH)
  print('**********************************************************************')
  print("Select Training Label Directory")
  LABEL_PATH = filedialog.askdirectory(title="Select Training Label Directory")
  print(LABEL_PATH)
  root.destroy
  if VAL == True:
    print('**********************************************************************')
    print("Select Validation Image Directory")
    VAL_PATH = filedialog.askdirectory(title="Select Validation Image Directory")
    print(VAL_PATH)
    print('**********************************************************************')
    print("Select Validation Label Directory")
    VAL_LABEL_PATH = filedialog.askdirectory(title="Select Validation Label Directory")
    print(VAL_LABEL_PATH)
    root.destroy
elif Select == 'Test':
  print("Select Testing Image Directory")
  TEST_PATH = filedialog.askdirectory(title="Select Testing Image Directory")
  print(TEST_PATH)
  root.destroy

In [ ]:
#@markdown ___
#@markdown ## **D. Resizing Images**
#@markdown

#@markdown Do You Want to Resize Your Images?

#@markdown *Downsizing your data can decrease training time and memory requirements.
#@markdown If you are using one of our pretrained models, (512x512) is required.*
#@markdown

Resize = False #@param  {type:"boolean"}

#@markdown Resize to MxN
M = 512 #@param {type:"integer"}
N = 512 #@param {type:"integer"}

#@markdown NOTE: *Resizing Operations Performed on Testing and Training Datasets Should be the Same.  If the Training Set was Resized Using ImageJ (for example) then the Test Set Should be Similarly Resized (rather than using this function).*

In [ ]:
#@markdown ___
#@markdown ## **E. Normalization**

#@markdown Conduct Normalization?
NORM = False #@param {type:"boolean"}

#@markdown *Useful when testing low contrast images*

#@markdown *May have unintended side effects on normal or high contrast images*

In [ ]:
#@markdown ___
#@markdown ## **F. Noise Detection and Removal**

#@markdown Would you like to conduct noise detection and removal (check for yes)?

#@markdown *For Test Images.  Makes Little Difference on Training Set.*
NR = False #@param {type: "boolean"}

if NR == True:
  #@markdown --- Select Path to Input Images (*Default is Test Image Path*)
  INPUT_IMG_PATH =  TEST_PATH #@param {type: "raw"}
  print(INPUT_IMG_PATH)
  #@markdown --- Select Path to Save DeNoised Images
  IMG_SAVE_PATH = "C:\\Users\\LIUY\\Desktop\\PythonCode\\images\\imagesDN" #@param {type: "string"}
  if not os.path.exists(IMG_SAVE_PATH):
    os.makedirs(IMG_SAVE_PATH)
  #@markdown --- Select Path to Save Noise Detection Maps
  NOISE_MAP = "C:\\Users\\LIUY\\Desktop\\PythonCode\\images\\Nmap" #@param {type:"string"}
  if not os.path.exists(NOISE_MAP):
    os.makedirs(NOISE_MAP)
  #@markdown

#@markdown **Do you Have UNET Installed?**
UNET = True #@param {type:'boolean'}
#@markdown --- If Yes, Provide the Path Below

#@markdown --- If No, Install to the Folder Below
UNET_PATH = "C:\\Users\\LIUY\\Desktop\\PythonCode\\unet" #@param {type: 'string'}
#@markdown

#@markdown --- Select UNet CNN Weights to use for Noise Detection.
NOISE_WEIGHTS = "Tissue" #@param ["Tissue", "Cell", "Select"] {type: "string"}
if NOISE_WEIGHTS == "Tissue":
  NW = r'C:\Users\LIUY\Desktop\PythonCode\weights\unet_noise_tissue.hdf5'
elif NOISE_WEIGHTS == "Cell":
  NW = r'C:\Users\LIUY\Desktop\PythonCode\weights\unet_noise_cell_line.hdf5'
elif NOISE_WEIGHTS == "Select":
  print("Use popup window to select weights")
  wp = filedialog.askopenfile(title=("Select Weights"))
  NW = wp.name

print(NW)

if NR == True:

  os.chdir(UNET_PATH)
  if UNET == False:
    !git clone --quiet https://github.com/zhixuhao/unet.git
    %cd unet
    UNET_PATH = os.path.join(UNET_PATH, "unet")

  from model import *
  from data import *
  import numpy as np 
  import cv2
  import os
  import glob
  import skimage.io as io
  import skimage.transform as trans

  model = load_model(NW)
  test_path = INPUT_IMG_PATH
  save_path = NOISE_MAP
  save_path2 = IMG_SAVE_PATH
  container = np.zeros((M,N,1,1));

  def bin_ndarray(ndarray, new_shape, operation='sum'):

    operation = operation.lower()
    if not operation in ['sum', 'mean']:
        raise ValueError("Operation not supported.")
    if ndarray.ndim != len(new_shape):
        raise ValueError("Shape mismatch: {} -> {}".format(ndarray.shape,
                                                           new_shape))
    compression_pairs = [(d, c//d) for d,c in zip(new_shape,
                                                  ndarray.shape)]
    flattened = [l for p in compression_pairs for l in p]
    ndarray = ndarray.reshape(flattened)
    for i in range(len(new_shape)):
        op = getattr(ndarray, operation)
        ndarray = op(-1*(i+1))
    return ndarray

  def image_normalized(file_path):

      img = cv2.imread(file_path,0)
      img_shape = img.shape
      image_size = (img_shape[1],img_shape[0])
      img_standard = bin_ndarray(img*1.2, (M,N), operation='mean')
      #img_standard = cv2.resize(img, (M, M), interpolation=cv2.INTER_CUBIC)
      img_new = img_standard
      imgT = img_standard
      img_new = np.asarray([img_new / 255.])
      return img_new,image_size, imgT

  for name in os.listdir(test_path):
    image_path = os.path.join(test_path,name)
    if os.path.isdir(image_path):
      continue
    ll = len(name)
    img,img_size, imgT = image_normalized(image_path)
    img = np.reshape(img,img.shape+(1,))
    results = model.predict(img)
    out = np.zeros(img.shape)

    out = 255*results[0,:,:,0];
    
    cv2.imwrite(os.path.join(save_path, ("%s") % (name[0:ll-3]+'png')), out)

    imgDN = imgT - out

    cv2.imwrite(os.path.join(save_path2, ("%s") % (name[0:ll-3]+'png')), imgDN)

    print(name)

In [ ]:
#@markdown ___
#@markdown ## **G. Process Data**
#@markdown Process Selected Image Directories
os.chdir(MRCNN_PATH)
def bin_ndarray(ndarray, new_shape, operation='sum'):
    """
    J.F. Sebastian
    Bins an ndarray in all axes based on the target shape, by summing or
        averaging.

    Number of output dimensions must match number of input dimensions and 
        new axes must divide old ones.

    Example
    -------
    >>> m = np.arange(0,100,1).reshape((10,10))
    >>> n = bin_ndarray(m, new_shape=(5,5), operation='sum')
    >>> print(n)

    [[ 22  30  38  46  54]
     [102 110 118 126 134]
     [182 190 198 206 214]
     [262 270 278 286 294]
     [342 350 358 366 374]]

    """
    operation = operation.lower()
    if not operation in ['sum', 'mean']:
        raise ValueError("Operation not supported.")
    if ndarray.ndim != len(new_shape):
        raise ValueError("Shape mismatch: {} -> {}".format(ndarray.shape,
                                                           new_shape))
    compression_pairs = [(d, c//d) for d,c in zip(new_shape,
                                                  ndarray.shape)]
    flattened = [l for p in compression_pairs for l in p]
    ndarray = ndarray.reshape(flattened)
    for i in range(len(new_shape)):
        op = getattr(ndarray, operation)
        ndarray = op(-1*(i+1))
    return ndarray

def reject_outliers(data, m=2):
    JoeGreen = np.mean(data)
    STD = np.std(data)
    MAX = np.max(data)
    data[(data) > (MAX-m*STD)] = 0
    return data

MP = 0
Num = 0
fin = 0

Spath = PROC_PATH
if NR == True:
  PATH = IMG_SAVE_PATH
else:
  if Select == "Train":
    PATH = TRAIN_PATH
  elif Select == "Test":
    PATH = TEST_PATH

#@markdown Do You Wish to Process and Write Your Data to PROC_PATH?
Write = True #@param {type:"boolean"}
NME = Select
while fin == 0:
  print(NME)
  for name in os.listdir(PATH):
      
    if Select == "Train":
      path = os.path.join(PATH, name)
      path2 = os.path.join(LABEL_PATH, name)
      if PATH == VAL_PATH:
        path = os.path.join(VAL_PATH, name)
        path2 = os.path.join(VAL_LABEL_PATH, name)
    elif Select == "Test":
      path = os.path.join(PATH, name)
      path2 = ''

    if os.path.isdir(path) or os.path.isdir(path2):
      continue

    print(name)
    ll = len(name)
    # Get Extension
    if Num == 0:
      nme, ext = os.path.splitext(name)

    if ext == '.tif':
      img = imread(path)
    elif ext == 'tiff':
      img = imread(path)
    else:
      img = cv2.imread(path,0)

    img = img.astype('uint8')

    # Normalize Images
    if NORM == True:
      img = reject_outliers(img, 2)
      img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)    
      #clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(35,35))
      #img = clahe.apply(img.astype('uint8'))
      #img = cv2.equalizeHist(img)

    # Resize
    if Resize == True:
      #img = cv2.resize(img,(M,N),interpolation=cv2.INTER_LINEAR)
      img = bin_ndarray(img*10, (M,N), operation='mean')

    if NORM == True:
      img = cv2.normalize(img, None, alpha=0, beta=180, norm_type=cv2.NORM_MINMAX)
      img = cv2.GaussianBlur(img,(3,3),0)

    # Show Images as They are Imported and Processed
    #cv2_imshow(img)
    sh = img.shape

    # Pixel Average
    if PATH != VAL_PATH:
      if len(sh) == 3 or 4:
        MP = (np.mean(img,axis=(0,1)) + MP)
      else:
        MP = (np.mean(img) + MP)

    Num = Num + 1

    # Write Images to Processed Folder
    if Write == True:
      if not os.path.exists(os.path.join(Spath, NME, name[0:ll-4], "images")):
        os.makedirs(os.path.join(Spath, NME, name[0:ll-4], "images"))
      if NME == "Val" and VAL == False:
        psimg = np.zeros(img.shape)
        cv2.imwrite(os.path.join(Spath, NME, name[0:ll-4], "images", name[0:ll-3]+'png'),psimg)
      else:
        cv2.imwrite(os.path.join(Spath, NME, name[0:ll-4], "images", name[0:ll-3]+'png'), img)
    
    if Select == "Train":
      if ext == '.tif' or ext == '.tiff':
        img2 = imread(path2)
      else:
        img2 = cv2.imread(path2)
      img2 = label(img2)
      img2 = cv2.resize(img2,(M,N),interpolation=cv2.INTER_NEAREST)
      P = img2.max()
      out = np.zeros([M, N, P])
      
      # Write Labels to Processed Folder
      if Write == True:
        if not os.path.exists(os.path.join(Spath, NME, name[0:ll-4], "masks")):		
          os.mkdir(os.path.join(Spath, NME, name[0:ll-4], "masks"))
        for n in range(1,P+1):
          ind = np.where(img2 == n)
          for i in range(0,ind[0].shape[0]-1):
            out[ind[0][i],ind[1][i],n-1] = 255
          cv2.imwrite(os.path.join(Spath, NME, name[0:ll-4], "masks", name[0:ll-4] + "_" + str(n-1) + ".png"),out[:,:,n-1])
      elif Write == True and VAL == False and PATH == VAL_PATH:
        if not os.path.exists(os.path.join(Spath, NME, name[0:ll-4], "masks")):		
          os.mkdir(os.path.join(Spath, NME, name[0:ll-4], "masks"))
        for n in range(1,2):
          ind = np.where(img2 == n)
          for i in range(0,ind[0].shape[0]-1):
            out[ind[0][i],ind[1][i],n-1] = 255
          cv2.imwrite(os.path.join(Spath, NME, name[0:ll-4], "masks", name[0:ll-4] + "_" + str(n-1) + ".png"),out[:,:,n-1])

    elif NME == "Val":
      #if VAL == True:
      if ext == '.tif' or ext == '.tiff':
        img2 = imread(path2)
      else:
        img2 = cv2.imread(path2)
      img2 = label(img2)
      img2 = cv2.resize(img2,(M,N),interpolation=cv2.INTER_NEAREST)
      P = img2.max()
      out = np.zeros([M, N, P])

      # Write Labels to Processed Folder
      if Write == True:
        if not os.path.exists(os.path.join(Spath, NME, name[0:ll-4], "masks")):		
          os.mkdir(os.path.join(Spath, NME, name[0:ll-4], "masks"))
        for n in range(1,P+1):
          ind = np.where(img2 == n)
          for i in range(0,ind[0].shape[0]-1):
            out[ind[0][i],ind[1][i],n-1] = 255
          cv2.imwrite(os.path.join(Spath, NME, name[0:ll-4], "masks", name[0:ll-4] + "_" + str(n-1) + ".png"),out[:,:,n-1])
      
      # elif VAL == False:
      #   img2 = np.ones([M,N])
      #   out2 = np.zeros([M, N, 1])
      #   if Write == True:
      #     if not os.path.exists(os.path.join(Spath, NME, name[0:ll-4], "masks")):		
      #       os.mkdir(os.path.join(Spath, NME, name[0:ll-4], "masks"))
      #     for n in range(1,2):
      #       ind = np.where(img2 == n)
      #       for i in range(0,ind[0].shape[0]-1):
      #         out[ind[0][i],ind[1][i],n-1] = 255
      #       cv2.imwrite(os.path.join(Spath, NME, name[0:ll-4], "masks", name[0:ll-4] + "_" + str(n-1) + ".png"),out2[:,:,n-1])


    elif Select == "Test":
      img2 = np.ones([M,N])
      out = np.zeros([M, N, 1])

      # Write Labels to Processed Folder
      if Write == True:
        if not os.path.exists(os.path.join(Spath, NME, name[0:ll-4], "masks")):		
          os.mkdir(os.path.join(Spath, NME, name[0:ll-4], "masks"))
        for n in range(1,2):
          ind = np.where(img2 == n)
          for i in range(0,ind[0].shape[0]-1):
            out[ind[0][i],ind[1][i],n-1] = 255
          cv2.imwrite(os.path.join(Spath, NME, name[0:ll-4], "masks", name[0:ll-4] + "_" + str(n-1) + ".png"),out[:,:,n-1])

  if NME == "Train":
    if VAL == True:
      PATH = VAL_PATH
    elif VAL == False:
      VTempI = os.path.join(PROC_PATH,"VTemp","images")
      VTempM = os.path.join(PROC_PATH,"VTemp","masks")
      os.makedirs(VTempI)
      os.makedirs(VTempM)
      cv2.imwrite(os.path.join(VTempI,"Val1.png"), np.ones(img.shape))
      cv2.imwrite(os.path.join(VTempM,"Val1.png"), np.ones(img.shape))
      VAL_PATH = VTempI
      VAL_LABEL_PATH = VTempM
      PATH = VAL_PATH
    NME = "Val"
  else:
    fin = 1
    if VAL == False:
      if os.path.exists(os.path.join(PROC_PATH,"VTemp")):
        shutil.rmtree(os.path.join(PROC_PATH,"VTemp"))

if Num == 0:
  Num = 1
MP = MP/Num
if len(sh) != 3 or 4:
  MP2 = np.array([MP,MP,MP])
else:
  MP2 = MP

# **III. Configuration**

In [ ]:
os.chdir(MRCNN_PATH)

class NucleusConfig(Config):
    """Configuration for training on the nucleus segmentation dataset."""
    # Give the configuration a recognizable name
    NAME = "nucleus"
    #@markdown ___
    #@markdown ## **Configuration Parameters**
    #@markdown --- Enter Desired Parameters or Leave Default Values

    #@markdown
    #@markdown ### **A. Batch size**
    #@markdown *Adjust depending on your GPU memory. Higher Number = Faster Runtime & Higher Memory Requirements*
    IMAGES_PER_GPU = 1  #@param {type:"integer"}

    #@markdown 
    #@markdown ### **B. Number of classes**
    #@markdown *Including background (e.g. background + nucleus = 2 classes)*
    NUM_CLASSES = 2  #@param {type:"integer"}

    #@markdown 
    #@markdown ### **C. Number of training and validation steps per epoch**
    EPOCHS = 10 #@param {type:"integer"}
    STEPS_PER_EPOCH = 20 #@param {type:"integer"}
    VALIDATION_STEPS =  10#@param {type:"integer"}

    #@markdown
    #@markdown ### **D. Confidence threshold for selecting between classes**
    #@markdown *Minimum CNN score a potential target must receive to be distinguished from the background.*
    DETECTION_MIN_CONFIDENCE = 0.7 #@param {type:"number"}

    #@markdown
    #@markdown ### **E. Detection Non Maximum Suppression (NMS) Threshold**
    #@markdown *Non-Maximum Suppression threshold for testing*
    DETECTION_NMS_THRESHOLD = 0.3 #@param {type:"number"}

    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    #@markdown
    #@markdown ### **F. Training RPN NMS Threshold**
    #@markdown *Region Proposal Network Non-Maximum Suppression threshold. You can increase this during training to generate more propsals.*
    RPN_NMS_THRESHOLD = 0.7 #@param {type:"number"}

    # Backbone network architecture
    # Supported values are: resnet50, resnet101
    #@markdown
    #@markdown ### **G. Network Backbone**
    #@markdown *Backbone network architecture.  Training and Testing must use the same value.*
    BACKBONE = "resnet50" #@param ["resnet50", "resnet101"] {type:"string"} 

    #@markdown
    #@markdown ### **H. Resize Input Images**
    #@markdown --- none: *No resizing or padding.*
    #@markdown --- crop: *Picks random crops from the image.*
    #@markdown --- square: *Resize and pad with zeros to get a square image of size [max_dim, max_dim].*
    #@markdown --- pad64:  *Pads width and height with zeros to make them multiples of 64 (test default).*
    IMAGE_RESIZE_MODE = "crop" #@param ["none", "crop", "square", "pad64"] {type:"string"}
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 1024
    IMAGE_MIN_SCALE = 2.0

    # Length of square anchor side in pixels
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)

    # ROIs kept after non-maximum supression (training and inference)
    POST_NMS_ROIS_TRAINING = 1000
    POST_NMS_ROIS_INFERENCE = 2000

    # How many anchors per image to use for RPN training
    RPN_TRAIN_ANCHORS_PER_IMAGE = 64

    # Image mean (RGB)
    MEAN_PIXEL = MP2
    #MEAN_PIXEL = np.array([43.53, 39.56, 48.22])

    # If enabled, resizes instance masks to a smaller size to reduce
    # memory load. Recommended when using high-resolution images.
    #@markdown 
    #@markdown ### **I. Mini Masks**
    #@markdown --- *Resizes instance masks to a smaller size to reduce memory load.* 
    #@markdown --- *Recommended when using high-resolution images.*
    USE_MINI_MASK = False #@param {type:"boolean"}
    #@markdown Length of mini mask side:
    length = 512 #@param {type:"integer"}
    MINI_MASK_SHAPE = (length, length)  # (height, width) of the mini-mask

    # Number of ROIs per image to feed to classifier/mask heads
    # The Mask RCNN paper uses 512 but often the RPN doesn't generate
    # enough positive proposals to fill this and keep a positive:negative
    # ratio of 1:3. You can increase the number of proposals by adjusting
    # the RPN NMS threshold.
    TRAIN_ROIS_PER_IMAGE = 256

    # Maximum number of ground truth instances to use in one image
    MAX_GT_INSTANCES = 256

    # Max number of final detections per image
    DETECTION_MAX_INSTANCES = 256

    # Learning rate and momentum
    # The Mask RCNN paper uses lr=0.02, but on TensorFlow it causes
    # weights to explode. Likely due to differences in optimizer
    # implementation.
    #@markdown
    #@markdown ### **J. Learning Rate**
    #@markdown --- *Lower values may take longer to acheive a lower loss but larger values may never settle (rec: 0.001-0.00001)*
    LEARNING_RATE = 0.001 #@param {type:"number"}
    LEARNING_MOMENTUM = 0.9

    # Weight decay regularization
    #@markdown --- *Learning rate decay (~1/10 * Learning Rate)*
    WEIGHT_DECAY = 0.0001 #@param {type:"number"}

class NucleusInferenceConfig(NucleusConfig):
    # Set batch size to 1 to run one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    # Don't resize imager for inferencing
    IMAGE_RESIZE_MODE = "pad64"
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7

config = NucleusConfig()
config.display()

# **IV. Define Functions**

In [ ]:
#@markdown ___
#@markdown ## **A. Define Classes and Load Images to the Network** 
#@markdown

class NucleusDataset(utils.Dataset):

    def load_nucleus(self, dataset_dir, subset):
        """Load a subset of the nuclei dataset.
        dataset_dir: Root directory of the dataset
        subset: Subset to load. The name of the sub-directory, such as:
                * train: images for training
                * val: validation images
                * test: test images
        """
        # Add classes. We have one class.
        # Naming the dataset nucleus, and the class nucleus
        #@markdown Dataset and Class Names
        #@markdown *Only 1 class, other than background.  Must adjust code and labels to add more classes.*
        DSET_NAMES = "nucleus" #@param {type: "string"}
        CLASS_NAME = "nucleus" #@param {type: "string"}
        self.add_class("nucleus", 1, "nucleus")

        # Which subset?
        # "val": select from data folder val
        # "train": use data from train folder
        # else: use the data from the specified sub-directory
        dataset_dir = os.path.join(dataset_dir, subset)
        image_ids = next(os.walk(dataset_dir))[1]

        # Add images
        for image_id in image_ids:
            self.add_image(
                "nucleus",
                image_id=image_id,
                path=os.path.join(dataset_dir, image_id, "images/{}.png".format(image_id)))

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        info = self.image_info[image_id]
        # Get mask directory from image path
        mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "masks")

        # Read mask files from .png image
        mask = []
        for f in next(os.walk(mask_dir))[2]:
            if ~os.path.isdir(f):
                m = skimage.io.imread(os.path.join(mask_dir, f)).astype(np.bool)
                mask.append(m)
        mask = np.stack(mask, axis=-1)
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID, we return an array of ones
        return mask, np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "nucleus":
            return info["id"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
#@markdown ___
#@markdown ## **B. Training Module**
#@markdown

def train(model, dataset_dir, subset):
    """Train the model."""
    # Training dataset.
    dataset_train = NucleusDataset()
    dataset_train.load_nucleus(dataset_dir, subset)
    dataset_train.prepare()

    # Validation dataset
    dataset_val = NucleusDataset()
    dataset_val.load_nucleus(dataset_dir, "Val")
    dataset_val.prepare()

    # Image augmentation
    # http://imgaug.readthedocs.io/en/latest/source/augmenters.html
    #@markdown ### **Select Image Augmentation Parameters**
    #@markdown Conduct Augmentation?
    AUG = True #@param  {type:"boolean"}
    #@markdown
    #@markdown ##### Fraction of Augmented Images to Flip Left/Right or Up/Down
    Flip_LR = 0.5 #@param {type: "number"}
    Flip_UD = 0.5 #@param {type: "number"}  
    #@markdown ##### Random Rotation Angles 
    ROT1 = 90  #@param {type: "number"}
    ROT2 = 180  #@param {type: "number"}
    ROT3 = 270  #@param {type: "number"}
    #@markdown ##### Random Scaling Low and High Value Range
    MULT_LO = 0.8  #@param {type: "number"}
    MULT_HI = 1.5  #@param {type: "number"}
    #@markdown ##### Low and High Range for Sigma Values for RandomBlurring
    SIG_LO = 0  #@param {type: "number"}
    SIG_HI = 5  #@param {type: "number"}
    #@markdown
    augmentation = iaa.SomeOf((0, 2), [
        iaa.Fliplr(Flip_LR),
        iaa.Flipud(Flip_UD),
        iaa.OneOf([iaa.Affine(rotate=ROT1),
                   iaa.Affine(rotate=ROT2),
                   iaa.Affine(rotate=ROT3)]),
        iaa.Multiply((MULT_LO, MULT_HI)),
        iaa.GaussianBlur(sigma=(SIG_LO, SIG_HI))
    ])
    if AUG == True:
      AUG_TYPE = augmentation
    else:
      AUG_TYPE = None
    #@markdown

    #@markdown Train Heads (Top Layers) or All Layers
    TRAIN_LAYERS = "all" #@param ["all", "heads"] {type:"string"}
    
    #@markdown *Training Using Multiple Learning Rates or Training on All Layers and then Heads can be done.  Double Click to Adjust Code.*
    
    # If starting from imagenet, train heads only for a bit
    # since they have random weights
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=config.EPOCHS,
                augmentation=augmentation,
                layers=TRAIN_LAYERS)
    
    # Add additional train sessions here, after the first, to continue training using different settings 
    #model.train(dataset_train, dataset_val,
     #       learning_rate=1e-4,
      #      epochs=EPOCHS+100,
       #     augmentation=augmentation,
        #    layers='heads')

In [ ]:
#@markdown ___
#@markdown ## **C. Detection Module**
#@markdown Prepare the test module

#@markdown

#@markdown ### **Conduct post-processing on predictions?**
#@markdown Overlap removal (experimental)
utc = pytz.utc
utc_dt = datetime.datetime.now()
eastern = timezone('US/Eastern')
loc_dt = utc_dt.astimezone(eastern)

import colorspacious
from glasbey import Glasbey
from skimage.color import label2rgb
import math
import codecs
from IPython.display import Image

color = np.array(([1,0,0],[0,1,0],[0,0,1],[1,1,0],[0,1,1],[1,0,1],[1,0.5,0],[0.5,1,0],[0,1,0.5],[0,0.5,1],[1,0,0.5],[0.5,0,1],[1,0.5,0.25],[0.25,0.5,1],[1,0.25,0.5],[0.5,0.25,1],[0.5,1,0.25],[0.25,1,0.5]),np.float32)
gb = Glasbey(base_palette=color, chroma_range = (60,100), no_black=True)
c4 = gb.generate_palette(size=18)
color4 = c4[1:]

def normalized(rgb):

        norm=np.zeros((512,512,3),np.float32)
        norm_rgb=np.zeros((512,512,3),np.uint8)

        b=rgb[:,:,0]
        g=rgb[:,:,1]
        r=rgb[:,:,2]

        sum=b+g+r

        norm[:,:,0]=b/sum*255.0
        norm[:,:,1]=g/sum*255.0
        norm[:,:,2]=r/sum*255.0

        norm_rgb=cv2.convertScaleAbs(norm)
        return norm_rgb

def overlay(mask, orig, clr):
  maskPR = label(mask)
  labels = label2rgb(label=maskPR, bg_label=0, bg_color=(0, 0, 0), colors=clr)
  L2 = normalized(labels)
  if len(orig.shape) < 3: 
    O2 = cv2.cvtColor(orig.astype('uint8'), cv2.COLOR_GRAY2BGR)
  else:
    O2 = orig
  comb = cv2.addWeighted(L2.astype('float64'),0.5,O2.astype('float64'),0.5,0)
  return comb

PROC = True #@param {type:"boolean"}
def detect(model, dataset_dir, subset, RESULTS_DIR):
    """Run detection on images in the given directory."""
    print("Running on {}".format(dataset_dir))
        
    # Read dataset
    dataset = NucleusDataset()
    dataset.load_nucleus(dataset_dir, subset)
    dataset.prepare()
    
    #@markdown
    #@markdown ### **Results Output Folder Name**
    #Out_Folder = 'Tissue_Nucleus'  + '_' +  loc_dt.strftime('%Y-%m-%d_%H:%M:%S_%Z%z') #@param {type: "raw"}
    Out_Folder = 'Tissue' #@param {type: "raw"}
    submit_dir = Out_Folder
    #print(submit_dir)
    tme = str(loc_dt.strftime('_%Y_%m_%d_%Hh-%Mm-%Ss'))
    submit_dir = os.path.join(RESULTS_DIR, submit_dir + tme)
    mask_dir = os.path.join(submit_dir, 'masks')
    print(mask_dir)
    if not os.path.exists(mask_dir):
      os.makedirs(mask_dir)

    # Load over images
    init = 0
    submission = []
    for image_id in dataset.image_ids:
        # Load image and run detection
        image = dataset.load_image(image_id)
        # Detect objects
        r = model.detect([image], verbose=0)[0]
        # Encode image to RLE. Returns a string of multiple lines
        source_id = dataset.image_info[image_id]["id"]
        rle = mask_to_rle(source_id, r["masks"], r["scores"])
        submission.append(rle)
        # Save image with masks
        visualize.display_instances(
            image, r['rois'], r['masks'], r['class_ids'],
            dataset.class_names, #r['scores'],
            show_bbox=False, show_mask=True,
            title="Predictions", captions = None)
        plt.savefig("{}/{}.png".format(submit_dir, dataset.image_info[image_id]["id"]))

        masks = r['masks'].astype(np.uint8)
        mask = np.zeros([masks.shape[0], masks.shape[1]], dtype='uint8')
        maskD = np.zeros([masks.shape[0], masks.shape[1]], dtype='uint8')
        diff = np.zeros([masks.shape[0], masks.shape[1]], dtype='uint8')
        props = np.zeros((masks.shape[2]))
        for n in range(0,masks.shape[2]):
            if PROC == False:
              mask = mask + (n+1)*masks[:,:,n]
            elif PROC == True:
              M2 = label(masks[:,:,n])
              props2 = regionprops(M2)
              for m in range(0,M2.max()):
                if props2[m].area < 100:
                  M2[M2==m+1] = 0
              M2[M2 > 0] = 1
              masks[:,:,n] = M2*masks[:,:,n]
              props2 = regionprops(masks[:,:,n])

              maskD = maskD + masks[:,:,n]
              
              if maskD.max() <= 1:
                mask = mask + (n+1)*masks[:,:,n]
              else:
                try:
                  diff[maskD > 1] = 1
                  diff2 = diff.copy()
                  pd = regionprops(diff)

                  area2 = props2[0].area 
                  aread = pd[0].area
                  Vals = diff*mask # Find value of existing region label, under new overlap
                  vals = Vals[Vals>0] # Not zero
                  vals = vals[vals != n+1] # Not the current label
                  vals = list(set(vals)) # Really should only be one left
                  props1 = regionprops(masks[:,:,vals[0]])
                  area1 = props1[0].area
                  div1 = aread/area1
                  div2 = aread/area2
                  dd = vals[0] + n+1
                  
                  mask = mask + (n+1)*masks[:,:,n]
                  if div1 < 0.15 and div2 < 0.15:
                      mask[diff > 0] = vals[0]
                  elif div1 < 0.15 and div2 > 0.15:
                      mask[diff > 0] = n+1
                      mask[mask==vals[0]] = n+1
                  elif div1 > 0.15 and div2 < 0.15:
                      mask[diff > 0] = vals[0]
                      mask[mask==n+1] = vals[0]
                  elif div1 > 0.15 and div2 > 0.15 and div1 < 0.6 and div2 < 0.6:

                      y0, x0 = pd[0].centroid
                      orientation = pd[0].orientation

                      x1 = x0 - math.sin(orientation) * 0.55 * pd[0].major_axis_length
                      y1 = y0 - math.cos(orientation) * 0.55 * pd[0].major_axis_length
                      x2 = x0 + math.sin(orientation) * 0.55 * pd[0].major_axis_length
                      y2 = y0 + math.cos(orientation) * 0.55 * pd[0].major_axis_length 

                      cv2.line(diff, (int(x2),int(y2)), (int(x0),int(y0)), (0, 0, 0), thickness=2)
                      cv2.line(diff, (int(x1),int(y1)), (int(x0),int(y0)), (0, 0, 0), thickness=2)

                      lbl1 = label(diff)
                      lbl1 = lbl1.astype('uint8')
                      cv2.line(lbl1, (int(x2),int(y2)), (int(x0),int(y0)), (1, 1, 1), thickness=2)
                      cv2.line(lbl1, (int(x1),int(y1)), (int(x0),int(y0)), (1, 1, 1), thickness=2)
                      lbl2 = lbl1*diff2
                      mask[lbl2 == 2] = n+1
                      mask[lbl2 == 1] = vals[0]
                                       
                  elif div1 > 0.6 or div2 > 0.6:
                    if area1 > area2:
                      mask[diff > 0] = vals[0]
                      mask[mask==n+1] = vals[0]
                    elif area2 > area1:
                      mask[diff > 0] = n+1
                      mask[mask==vals[0]] = n+1
                except Exception:
                  continue
              maskD[maskD > 1] = 1
              diff = np.zeros([masks.shape[0], masks.shape[1]], dtype='uint8')

        #print(dataset_dir+ '/' + dataset.image_info[image_id]["id"])
        cv2.imwrite(submit_dir + '/masks/' + dataset.image_info[image_id]["id"] + '.png', mask)
        ovr = overlay(mask, image, color4)
        cv2.imwrite(submit_dir + '/' + dataset.image_info[image_id]["id"] + '.png', ovr)
        if PROC:
          print("Processed Overlay")
          _, ax = plt.subplots(1, figsize=(16, 16))
          ax.axis('off')
          plt.imshow(ovr.astype('uint8'),clim=(0.0, 1.0))
          plt.show()
          init = init + 1

    # Save to csv file
    submission = "ImageId,EncodedPixels\n" + "\n".join(submission)
    file_path = os.path.join(submit_dir, "submit.csv")
    with open(file_path, "w") as f:
        f.write(submission)
    print("Saved to ", submit_dir)

    return submit_dir

In [ ]:
#@markdown ___
#@markdown ## **D. RLE Encoding and Decoding**
#@markdown

def rle_encode(mask):
    """Encodes a mask in Run Length Encoding (RLE).
    Returns a string of space-separated values.
    """
    assert mask.ndim == 2, "Mask must be of shape [Height, Width]"
    # Flatten it column wise
    m = mask.T.flatten()
    # Compute gradient. Equals 1 or -1 at transition points
    g = np.diff(np.concatenate([[0], m, [0]]), n=1)
    # 1-based indicies of transition points (where gradient != 0)
    rle = np.where(g != 0)[0].reshape([-1, 2]) + 1
    # Convert second index in each pair to lenth
    rle[:, 1] = rle[:, 1] - rle[:, 0]
    return " ".join(map(str, rle.flatten()))


def rle_decode(rle, shape):
    """Decodes an RLE encoded list of space separated
    numbers and returns a binary mask."""
    rle = list(map(int, rle.split()))
    rle = np.array(rle, dtype=np.int32).reshape([-1, 2])
    rle[:, 1] += rle[:, 0]
    rle -= 1
    mask = np.zeros([shape[0] * shape[1]], np.bool)
    for s, e in rle:
        assert 0 <= s < mask.shape[0]
        assert 1 <= e <= mask.shape[0], "shape: {}  s {}  e {}".format(shape, s, e)
        mask[s:e] = 1
    # Reshape and transpose
    mask = mask.reshape([shape[1], shape[0]]).T
    return mask


def mask_to_rle(image_id, mask, scores):
    "Encodes instance masks to submission format."
    assert mask.ndim == 3, "Mask must be [H, W, count]"
    # If mask is empty, return line with image ID only
    if mask.shape[-1] == 0:
        return "{},".format(image_id)
    # Remove mask overlaps
    # Multiply each instance mask by its score order
    # then take the maximum across the last dimension
    order = np.argsort(scores)[::-1] + 1  # 1-based descending
    mask = np.max(mask * np.reshape(order, [1, 1, -1]), -1)
    # Loop over instance masks
    lines = []
    for o in order:
        m = np.where(mask == o, 1, 0)
        # Skip if empty
        if m.sum() == 0.0:
            continue
        rle = rle_encode(m)
        lines.append("{}, {}".format(image_id, rle))
    return "\n".join(lines)

# **V. Run the Network**

In [ ]:
#@markdown ___
#@markdown ## **A. Select Weights**

os.chdir(MRCNN_PATH)

ROOT_DIR = MRCNN_PATH

# Select training/testing weights

#@markdown Select pre-trained weights for testing or to continue training on.
#@markdown
#@markdown Choose the "Select" option to upload other weights.
Weights = "coco" #@param ["coco", "imagenet", "Kaggle", "Storm_Tissue", "Storm_Cell", "Select"] {type:"string"}
if Weights == "coco":
  weights_path = r'C:\Users\LIUY\Desktop\PythonCode\weights\mask_rcnn_coco.h5'
elif Weights == "imagenet":
  weights_path = model.get_imagenet_weights()
elif Weights == "Kaggle":
  weights_path = r'C:\Users\LIUY\Desktop\PythonCode\weights\mask_rcnn_kaggle_v1.h5'
elif Weights == "Storm_Tissue":
  weights_path = r'C:\Users\LIUY\Desktop\PythonCode\weights\mask_rcnn_nucleus_tissue.h5'
elif Weights == "Storm_Cell":
  weights_path = r'C:\Users\LIUY\Desktop\PythonCode\weights\mask_rcnn_nucleus_cell.h5'
elif Weights == "Select":
  print('Use popup window to select weights')
  weights = filedialog.askopenfile(title=('Select Weights'))
  weights_path = weights.name

print(weights_path) 
#@markdown

#@markdown ### **Select Network Training Log Directory**
#@markdown Where your newly trained weights will be saved.
DEFAULT_LOGS_DIR = 'C:\\Users\\LIUY\\Desktop\\PythonCode\\Mask_RCNN\\logs' #@param {type: "string"}
if not os.path.exists(DEFAULT_LOGS_DIR):
  os.makedirs(DEFAULT_LOGS_DIR)
#@markdown

#@markdown ### **Select Segmentation Results Directory**
#@markdown Where your test results will be saved
RESULTS_DIR = 'C:\\Users\\LIUY\\Desktop\\PythonCode\\Mask_RCNN\\Results' #@param {type: "string"}
if not os.path.exists(RESULTS_DIR):
  os.makedirs(RESULTS_DIR)


In [ ]:
#@markdown ___
#@markdown ## **B. Run Network**

#@markdown ### Test or Train?
command = 'train' #@param ["test", "train"] {type: "string"}

# Configurations
if command == "train":
    config = NucleusConfig()
    log_dir = DEFAULT_LOGS_DIR
    #if not os.path.exists(weights_path):
     # os.makedirs(weights_path)
else:
    config = NucleusInferenceConfig()
#config.display()

print(weights_path)
# Create model
if command == "train":
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=log_dir)
else:
    model = modellib.MaskRCNN(mode="inference", config=config,
                              model_dir=weights_path)
    
if Weights == "coco":
  model.load_weights(weights_path, by_name=True, exclude=[
  "mrcnn_class_logits", "mrcnn_bbox_fc",
  "mrcnn_bbox", "mrcnn_mask"])
else:
  model.load_weights(weights_path, by_name=True)

# Train or evaluate
if command == "train":
    train(model, PROC_PATH, subset='Train')
elif command == "test":
    sdir = detect(model, PROC_PATH, RESULTS_DIR=RESULTS_DIR, subset='Test',)
else:
    print("'{}' is not recognized. "
          "Use 'train' or 'detect'".format(command))